In [1]:

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

from sklearn.metrics import classification_report
from transformers import AutoTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, concatenate_datasets
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

log_file = 'ml_nli.txt'

# ds1 = load_dataset('MoritzLaurer/multilingual-NLI-26lang-2mil7', split='ar_anli') 
# ds2 = load_dataset('MoritzLaurer/multilingual-NLI-26lang-2mil7', split='ar_fever')
# ds3 = load_dataset('MoritzLaurer/multilingual-NLI-26lang-2mil7', split='ar_ling')
# ds4 = load_dataset('MoritzLaurer/multilingual-NLI-26lang-2mil7', split='ar_mnli')
# ds5 = load_dataset('MoritzLaurer/multilingual-NLI-26lang-2mil7', split='ar_wanli')
# # display(ds)


ds1 = pd.read_parquet('multi_nli/ar_anli-00000-of-00001-d5ddcd7a96189c94.parquet')
ds2 = pd.read_parquet('multi_nli/ar_fever-00000-of-00001-75e864b8c1cf8d17.parquet')
ds3 = pd.read_parquet('multi_nli/ar_ling-00000-of-00001-f4e042f46b091cf7.parquet')
ds4 = pd.read_parquet('multi_nli/ar_mnli-00000-of-00001-13deaea9065575d9.parquet')
ds5 = pd.read_parquet('multi_nli/ar_wanli-00000-of-00001-7f580e9d2eff0880.parquet')

ds1 = Dataset.from_pandas(ds1)
ds2 = Dataset.from_pandas(ds2)
ds3 = Dataset.from_pandas(ds3)
ds4 = Dataset.from_pandas(ds4)
ds5 = Dataset.from_pandas(ds5)

ds1 = ds1.train_test_split(test_size=0.2)
ds2 = ds2.train_test_split(test_size=0.2)
ds3 = ds3.train_test_split(test_size=0.2)
ds4 = ds4.train_test_split(test_size=0.2)
ds5 = ds5.train_test_split(test_size=0.2)

# display(ds4)

dataset_t = concatenate_datasets([ds1['train'], ds2['train'], ds3['train'], ds4['train'], ds5['train']])
dataset_v = concatenate_datasets([ds1['test'], ds2['test'], ds3['test'], ds4['test'], ds5['test']])

# display(dataset_train)
display(len(dataset_t))
display(len(dataset_v))

unique_labels = set(dataset_t['label'])
classes_num = len(unique_labels)

print(f'Unique Labels: {unique_labels}')
print(f'Number of Classes: {classes_num}')

models = ['faisalq/bert-base-arabic-wordpiece', 'faisalq/bert-base-arabic-senpiece',
                'faisalq/bert-base-arabic-bbpe']

with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')

for model_name in models:
    for i in range(3):
        print(f'{model_name}, try:{i}')
        dataset_train = dataset_t
        dataset_validation = dataset_v
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = BertForSequenceClassification.from_pretrained(model_name,
                                                              num_labels=classes_num).to('cuda')                                                 
                                                     
        
        max_length = 128
        
        def preprocess_function(examples):
            return tokenizer(examples['premise'], examples['hypothesis'], truncation=True, padding="max_length",
                            max_length=max_length)
        
        
        dataset_train = dataset_train.map(preprocess_function, batched=True)
        dataset_validation = dataset_validation.map(preprocess_function, batched=True)

                
        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)    
            acc = accuracy_score(labels, predictions)        
            f1 = f1_score(labels, predictions, average='macro')   
            with open(log_file, 'a') as f:
                f.write(f'{model_name},{acc},{f1}\n')
            return {'accuracy': acc, 'f1_score': f1}
        

        epochs = 10
        save_steps = 10000 #save checkpoint every 10000 steps
        batch_size = 256
        
        
        training_args = TrainingArguments(
            output_dir = 'bert/',
            overwrite_output_dir=True,
            num_train_epochs = epochs,
            per_device_train_batch_size = batch_size,
            per_device_eval_batch_size = batch_size,
            save_steps = save_steps,
            save_total_limit = 1, 
            fp16=True,
            learning_rate = 5e-5,  # 5e-5 is the default
            # weight_decay=0.01,
            logging_steps = 120, #50_000
            evaluation_strategy = 'steps',
            eval_steps = 120
            
        )
        
        
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=dataset_train,
            eval_dataset=dataset_validation,
            compute_metrics=compute_metrics,
        )
        
        trainer.train()
        # trainer.evaluate()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results.to_csv('mlnli_results.csv')
display(best_results)

2024-02-11 06:39:16.257815: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-11 06:39:16.282131: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-11 06:39:16.718589: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


84000

21000

Unique Labels: {0, 1, 2}
Number of Classes: 3
faisalq/bert-base-arabic-wordpiece, try:0


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at faisalq/bert-base-arabic-wordpiece and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Map:   0%|          | 0/84000 [00:00<?, ? examples/s]

Map:   0%|          | 0/21000 [00:00<?, ? examples/s]

/home/ffq/miniconda3/envs/p1/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1 Score
120,0.992300,0.891256,0.580333,0.567090
240,0.873900,0.830963,0.619619,0.605763
360,0.805600,0.821265,0.636048,0.621276
480,0.712100,0.809452,0.644810,0.633600
600,0.704800,0.805529,0.647048,0.636318
720,0.611600,0.868566,0.651429,0.643421
840,0.526400,0.879853,0.647905,0.642287
960,0.527100,0.866407,0.651619,0.642910
1080,0.398800,1.044992,0.645810,0.639966
1200,0.364200,0.981877,0.654619,0.647436


,Model,Accuracy,F1
0,faisalq/bert-base-arabic-bbpe,0.654619,0.647436
1,faisalq/bert-base-arabic-bbpe,0.654619,0.647436
2,faisalq/bert-base-arabic-bbpe,0.654619,0.647436
3,faisalq/bert-base-arabic-senpiece,0.668667,0.661311
4,faisalq/bert-base-arabic-senpiece,0.668667,0.661311
5,faisalq/bert-base-arabic-senpiece,0.668667,0.661311
6,faisalq/bert-base-arabic-wordpiece,0.664571,0.658136
